# Enunciado do Teste:

SLA (Service Level Agreement) é uma métrica que indica se um chamado fechado foi atendido dentro do tempo contratado ou não.
 
A base de dados fornecida representa um conjunto de chamados unitários, abertos e fechados para 10 clientes diferentes (coluna customerCode). O SLA no instante t é calculado pelo número de chamados fechados dentro mês até o instante t (coluna “onTimeSolution=S”) dividido pelo total de chamados do mês até o instante t.
 
O desafio proposto é gerar um modelo preditivo que possa informar a percentagem de SLA no mês para cada cliente.  
Os dados fornecidos são de 01 de Janeiro de 2019 até 26 de Fevereiro de 2019. O modelo deve então prever qual será o SLA do dia 28 de Fevereiro (final do mês). 

## Considerações:

*	Para identificar qual ticket já foi fechado, usa-se a coluna “callStatus”, onde os status fechados são: N0, N4 e CV.
*	Para identificar quais tickets foram fechados dentro ou fora do SLA, usa-se a coluna “onTimeSolution”(S = foi fechado dentro e N = não foi fechado dentro)

### Carregamentos dos pacotes & Definição de funções

In [4]:
import pandas as pd
import numpy as np
from datetime import date

In [5]:
# Visualização inicial da tabela de dados
df00 = pd.read_csv("ticket_cientista.csv", sep = ";", low_memory = False)
df00.head()

,attendanceType,attendanceTypeId,averageRepairTime,averageRepairTimeType,averageServiceTime,averageSolutionTime,averageWaitingTimeType,callBranch,callBranchName,callCloseDate,...,repairTimeEngineer,schedulingDate,schedulingDateTime,schedulingTime,segmentId,siteCity,siteName,siteState,slaStatus,type
0,1,5,0,35.0,222,222,52.0,86,Natal,NaN,...,0.0,20180207,"ISODate(""2018-02-07T15:00:00.000-02:00"")",15:00,35,CAICÓ,LEXBRA000010380,RN,NaN,U
1,0,2,164,27.0,131,295,66.0,52,Campinas,20190222.0,...,164.0,NaN,NaN,NaN,35,PIRACICABA,LEXMARK00013169,SP,N,U
2,0,2,19,27.0,264,283,66.0,52,Campinas,20190222.0,...,19.0,NaN,NaN,NaN,35,JUNDIAI,LEXMARK00013402,SP,N,U
3,0,2,360,56.0,257,617,67.0,70,Paissandu,NaN,...,0.0,NaN,NaN,NaN,35,SAO PAULO,LEXMARK00004162,SP,S,U
4,0,2,63,21.0,146,209,55.0,AD,Campo Grande,NaN,...,0.0,NaN,NaN,NaN,35,CAMPO GRANDE,LEXMARK00012469,MS,N,U


In [6]:
# Visualização das colunas da tabela
df00.columns

Index(['attendanceType', 'attendanceTypeId', 'averageRepairTime',
       'averageRepairTimeType', 'averageServiceTime', 'averageSolutionTime',
       'averageWaitingTimeType', 'callBranch', 'callBranchName',
       'callCloseDate', 'callCloseTime', 'callNumber', 'callOpenDate',
       'callOpenTime', 'callRegion', 'callRegional', 'callRegionalName',
       'callRegionName', 'callStatus', 'closeDateTime', 'currentDateTime',
       'customerCode', 'deadline', 'deadlineDateTime', 'deadlineSolution',
       'engineerCode', 'equipmentType', 'onTimeSolution', 'openDateTime',
       'priority', 'productCode', 'rawCurrentDateTime', 'received_ts',
       'released_parts', 'repairTimeEngineer', 'schedulingDate',
       'schedulingDateTime', 'schedulingTime', 'segmentId', 'siteCity',
       'siteName', 'siteState', 'slaStatus', 'type'],
      dtype='object')

In [7]:
# Selecação das variáveis para uso no modelo de série temporal e média móvel
# Como a Série Temporal é um modelo que envolve a predição e a modelagem do próprio target nós só precisamos construir uma variável
# que simbolize o que desejamos projetar, neste caso, o percentual de SLA's num determinado período por tipo de cliente
df01 = df00[["customerCode", "callOpenDate", "callCloseDate", "callStatus", "onTimeSolution"]]
df01.head()

,customerCode,callOpenDate,callCloseDate,callStatus,onTimeSolution
0,215,20180207,NaN,AT,S
1,215,20190221,20190222.0,N4,S
2,215,20190221,20190222.0,N4,S
3,215,20190221,NaN,AT,S
4,215,20190221,NaN,N7,S


### Criação de uma variável temporal para indicar a referência do chamado e de uma flag para indicar se o chamado foi fechado dentro do SLA

In [8]:
# Vamos definir uma variável chamada CALL_DATREF_INFO que nos diz a referência em que o chamado foi ou aberto(e permanece aberto) ou fechado,
# isso nos ajudará a ter uma variável temporal para realizar contagens e agrupamentos

val_callopndate = list(df01["callOpenDate"])
val_callclsdate = list(df01["callCloseDate"])
val_callstats   = list(df01["callStatus"])
val_ontimesol   = list(df01["onTimeSolution"])

flag_clsin         = []
val_calldatref     = []
val_calldatref_mon = []

for i, c_st in enumerate(val_callstats):
    
    # Verificando se os chamados estão fechados, se sim atribuimos o valor da variável "callCloseDate", caso contrário
    # atribuiremos o valor da variável "callOpenDate", ambas no formato inteiro AAAAMMDD
    if c_st in ["N0", "N4", "CV"]:
        val_calldatref.append(int(val_callclsdate[i]))
        val_calldatref_mon.append(int(val_callclsdate[i]/100))
        
        if val_ontimesol[i] == "S":
            flag_clsin.append(1)
        else:
            flag_clsin.append(0)
    else:
        val_calldatref.append(int(val_callopndate[i]))
        val_calldatref_mon.append(int(val_callopndate[i]/100))
        flag_clsin.append(0)
        
call_datref_info_df = pd.DataFrame(val_calldatref, columns = ["CALL_DATREF_INFO"])
call_datref_mon_df  = pd.DataFrame(val_calldatref_mon, columns = ["CALL_DATREF_MON"])
flag_clsin_df       = pd.DataFrame(flag_clsin, columns = ["FLAG_SLA"])

In [9]:
# Trazendo as novas colunas para o data frame
df02 = pd.merge(df01, call_datref_info_df, left_index = True, right_index = True)
df03 = pd.merge(df02, call_datref_mon_df, left_index = True, right_index = True)
df04 = pd.merge(df03, flag_clsin_df, left_index = True, right_index = True)
df04.head()

,customerCode,callOpenDate,callCloseDate,callStatus,onTimeSolution,CALL_DATREF_INFO,CALL_DATREF_MON,FLAG_SLA
0,215,20180207,NaN,AT,S,20180207,201802,0
1,215,20190221,20190222.0,N4,S,20190222,201902,1
2,215,20190221,20190222.0,N4,S,20190222,201902,1
3,215,20190221,NaN,AT,S,20190221,201902,0
4,215,20190221,NaN,N7,S,20190221,201902,0


In [10]:
# Ordenando os valores por "customerCode" e "CALL_DATREF_INFO" e dropando algumas variáveis
df04.sort_values(["customerCode", "CALL_DATREF_INFO"], ascending = [True, True], inplace = True)
df04.drop(["callOpenDate", "callCloseDate", "callStatus", "onTimeSolution"], axis = 1, inplace = True)
df04.head()

,customerCode,CALL_DATREF_INFO,CALL_DATREF_MON,FLAG_SLA
0,215,20180207,201802,0
2081,215,20180808,201808,0
2206,215,20180809,201808,0
3114,215,20180816,201808,0
3489,215,20180821,201808,0


In [11]:
# Vamos splitar nossos dados em 10 data frames distintos referentes a cada um dos 10 clientes
# e filtrando os meses para que estejam entre Jan/19 a Fev/19
np.unique(df04["customerCode"])

array([   215,   2581,   3363,   8068,  39159,  87520, 372301, 797391,
       900102, 900995], dtype=int64)

In [12]:
df_clie01 = df04[(df04["customerCode"] == 215)    & ((df04["CALL_DATREF_MON"] == 201901) | (df04["CALL_DATREF_MON"] == 201902))]
df_clie02 = df04[(df04["customerCode"] == 2581)   & ((df04["CALL_DATREF_MON"] == 201901) | (df04["CALL_DATREF_MON"] == 201902))]
df_clie03 = df04[(df04["customerCode"] == 3363)   & ((df04["CALL_DATREF_MON"] == 201901) | (df04["CALL_DATREF_MON"] == 201902))]
df_clie04 = df04[(df04["customerCode"] == 8068)   & ((df04["CALL_DATREF_MON"] == 201901) | (df04["CALL_DATREF_MON"] == 201902))]
df_clie05 = df04[(df04["customerCode"] == 39159)  & ((df04["CALL_DATREF_MON"] == 201901) | (df04["CALL_DATREF_MON"] == 201902))]
df_clie06 = df04[(df04["customerCode"] == 87520)  & ((df04["CALL_DATREF_MON"] == 201901) | (df04["CALL_DATREF_MON"] == 201902))]
df_clie07 = df04[(df04["customerCode"] == 372301) & ((df04["CALL_DATREF_MON"] == 201901) | (df04["CALL_DATREF_MON"] == 201902))]
df_clie08 = df04[(df04["customerCode"] == 797391) & ((df04["CALL_DATREF_MON"] == 201901) | (df04["CALL_DATREF_MON"] == 201902))]
df_clie09 = df04[(df04["customerCode"] == 900102) & ((df04["CALL_DATREF_MON"] == 201901) | (df04["CALL_DATREF_MON"] == 201902))]
df_clie10 = df04[(df04["customerCode"] == 900995) & ((df04["CALL_DATREF_MON"] == 201901) | (df04["CALL_DATREF_MON"] == 201902))]

### Vamos fazer uma soma cumulativa dos chamados fechados dentro do SLA durante os dias de cada mês levando em conta cada cliente separadamente

### Cliente 01 - ID: 215

In [13]:
# Cálculo da quantidade de chamados fechados dentro do SLA e o total de chamados por dia
slas_fechados     = df_clie01["FLAG_SLA"].groupby(df_clie01["CALL_DATREF_INFO"]).sum()
total_de_calls    = df_clie01["FLAG_SLA"].groupby(df_clie01["CALL_DATREF_INFO"]).count()
slas_fechados_df  = pd.DataFrame(slas_fechados.values, columns = ["NUM_CLOSED_SLAS"], index = slas_fechados.index)
total_de_calls_df = pd.DataFrame(total_de_calls.values, columns = ["TOTAL_CALLS"], index = total_de_calls.index)

df_aux    = pd.merge(slas_fechados_df, total_de_calls_df, left_index = True, right_index = True)

df_clie01.set_index("CALL_DATREF_INFO", drop = True, inplace = True)
df_clie01.drop("FLAG_SLA", axis = 1, inplace = True)

df_clie01_ = pd.merge(df_clie01, df_aux, left_on = "CALL_DATREF_INFO", right_on = "CALL_DATREF_INFO")
df_clie01_.drop_duplicates(inplace = True)
df_clie01_.head()

C:\Users\Thiago\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,customerCode,CALL_DATREF_MON,NUM_CLOSED_SLAS,TOTAL_CALLS
CALL_DATREF_INFO,,,,
20190102,215,201901,135,179
20190103,215,201901,133,157
20190104,215,201901,138,168
20190105,215,201901,15,18
20190106,215,201901,4,6


In [14]:
# Criação do percentual cumulativo de SLAS até o instante t
slas_fechados_cum  = df_clie01_["NUM_CLOSED_SLAS"].groupby(df_clie01_["CALL_DATREF_MON"]).cumsum()
total_de_calls_cum = df_clie01_["TOTAL_CALLS"].groupby(df_clie01_["CALL_DATREF_MON"]).cumsum()
perc_sla_cum       = (slas_fechados_cum / total_de_calls_cum) * 100
perc_sla_cum_df    = pd.DataFrame(perc_sla_cum.values, columns = ["PERC_SLAS_CUM"], index = perc_sla_cum.index)

df_clie01_ = pd.merge(df_clie01_, perc_sla_cum_df, left_index = True, right_index = True)
df_clie01_.head()

,customerCode,CALL_DATREF_MON,NUM_CLOSED_SLAS,TOTAL_CALLS,PERC_SLAS_CUM
CALL_DATREF_INFO,,,,,
20190102,215,201901,135,179,75.418994
20190103,215,201901,133,157,79.761905
20190104,215,201901,138,168,80.555556
20190105,215,201901,15,18,80.651341
20190106,215,201901,4,6,80.492424


In [15]:
# Criação da ABT de modelagem
ABT_clie01 = df_clie01_.drop(["customerCode", "CALL_DATREF_MON", "NUM_CLOSED_SLAS", "TOTAL_CALLS"], axis = 1)
ABT_clie01.head()

,PERC_SLAS_CUM
CALL_DATREF_INFO,
20190102,75.418994
20190103,79.761905
20190104,80.555556
20190105,80.651341
20190106,80.492424


### Cliente 02 - ID: 2581

In [16]:
# Cálculo da quantidade de chamados fechados dentro do SLA e o total de chamados por dia
slas_fechados     = df_clie02["FLAG_SLA"].groupby(df_clie02["CALL_DATREF_INFO"]).sum()
total_de_calls    = df_clie02["FLAG_SLA"].groupby(df_clie02["CALL_DATREF_INFO"]).count()
slas_fechados_df  = pd.DataFrame(slas_fechados.values, columns = ["NUM_CLOSED_SLAS"], index = slas_fechados.index)
total_de_calls_df = pd.DataFrame(total_de_calls.values, columns = ["TOTAL_CALLS"], index = total_de_calls.index)

df_aux     = pd.merge(slas_fechados_df, total_de_calls_df, left_index = True, right_index = True)

df_clie02.set_index("CALL_DATREF_INFO", drop = True, inplace = True)
df_clie02.drop("FLAG_SLA", axis = 1, inplace = True)

df_clie02_ = pd.merge(df_clie02, df_aux, left_on = "CALL_DATREF_INFO", right_on = "CALL_DATREF_INFO")
df_clie02_.drop_duplicates(inplace = True)

# Criação do percentual cumulativo de SLAS até o instante t
slas_fechados_cum  = df_clie02_["NUM_CLOSED_SLAS"].groupby(df_clie02_["CALL_DATREF_MON"]).cumsum()
total_de_calls_cum = df_clie02_["TOTAL_CALLS"].groupby(df_clie02_["CALL_DATREF_MON"]).cumsum()
perc_sla_cum       = (slas_fechados_cum / total_de_calls_cum) * 100
perc_sla_cum_df    = pd.DataFrame(perc_sla_cum.values, columns = ["PERC_SLAS_CUM"], index = perc_sla_cum.index)

df_clie02_ = pd.merge(df_clie02_, perc_sla_cum_df, left_index = True, right_index = True)

# Criação da ABT de modelagem
ABT_clie02 = df_clie02_.drop(["customerCode", "CALL_DATREF_MON", "NUM_CLOSED_SLAS", "TOTAL_CALLS"], axis = 1)
ABT_clie02.head()

,PERC_SLAS_CUM
CALL_DATREF_INFO,
20190102,95.238095
20190103,97.142857
20190104,93.406593
20190107,95.000000
20190108,95.714286


### Cliente 03 - ID: 3363

In [17]:
# Cálculo da quantidade de chamados fechados dentro do SLA e o total de chamados por dia
slas_fechados     = df_clie03["FLAG_SLA"].groupby(df_clie03["CALL_DATREF_INFO"]).sum()
total_de_calls    = df_clie03["FLAG_SLA"].groupby(df_clie03["CALL_DATREF_INFO"]).count()
slas_fechados_df  = pd.DataFrame(slas_fechados.values, columns = ["NUM_CLOSED_SLAS"], index = slas_fechados.index)
total_de_calls_df = pd.DataFrame(total_de_calls.values, columns = ["TOTAL_CALLS"], index = total_de_calls.index)

df_aux     = pd.merge(slas_fechados_df, total_de_calls_df, left_index = True, right_index = True)

df_clie03.set_index("CALL_DATREF_INFO", drop = True, inplace = True)
df_clie03.drop("FLAG_SLA", axis = 1, inplace = True)

df_clie03_ = pd.merge(df_clie03, df_aux, left_on = "CALL_DATREF_INFO", right_on = "CALL_DATREF_INFO")
df_clie03_.drop_duplicates(inplace = True)

# Criação do percentual cumulativo de SLAS até o instante t
slas_fechados_cum  = df_clie03_["NUM_CLOSED_SLAS"].groupby(df_clie03_["CALL_DATREF_MON"]).cumsum()
total_de_calls_cum = df_clie03_["TOTAL_CALLS"].groupby(df_clie03_["CALL_DATREF_MON"]).cumsum()
perc_sla_cum       = (slas_fechados_cum / total_de_calls_cum) * 100
perc_sla_cum_df    = pd.DataFrame(perc_sla_cum.values, columns = ["PERC_SLAS_CUM"], index = perc_sla_cum.index)

df_clie03_ = pd.merge(df_clie03_, perc_sla_cum_df, left_index = True, right_index = True)

# Criação da ABT de modelagem
ABT_clie03 = df_clie03_.drop(["customerCode", "CALL_DATREF_MON", "NUM_CLOSED_SLAS", "TOTAL_CALLS"], axis = 1)
ABT_clie03.head()

,PERC_SLAS_CUM
CALL_DATREF_INFO,
20190102,94.074074
20190103,94.795539
20190104,95.854922
20190105,95.959596
20190106,95.989975


### Cliente 04 - ID: 8068

In [18]:
# Cálculo da quantidade de chamados fechados dentro do SLA e o total de chamados por dia
slas_fechados     = df_clie04["FLAG_SLA"].groupby(df_clie04["CALL_DATREF_INFO"]).sum()
total_de_calls    = df_clie04["FLAG_SLA"].groupby(df_clie04["CALL_DATREF_INFO"]).count()
slas_fechados_df  = pd.DataFrame(slas_fechados.values, columns = ["NUM_CLOSED_SLAS"], index = slas_fechados.index)
total_de_calls_df = pd.DataFrame(total_de_calls.values, columns = ["TOTAL_CALLS"], index = total_de_calls.index)

df_aux     = pd.merge(slas_fechados_df, total_de_calls_df, left_index = True, right_index = True)

df_clie04.set_index("CALL_DATREF_INFO", drop = True, inplace = True)
df_clie04.drop("FLAG_SLA", axis = 1, inplace = True)

df_clie04_ = pd.merge(df_clie04, df_aux, left_on = "CALL_DATREF_INFO", right_on = "CALL_DATREF_INFO")
df_clie04_.drop_duplicates(inplace = True)

# Criação do percentual cumulativo de SLAS até o instante t
slas_fechados_cum  = df_clie04_["NUM_CLOSED_SLAS"].groupby(df_clie04_["CALL_DATREF_MON"]).cumsum()
total_de_calls_cum = df_clie04_["TOTAL_CALLS"].groupby(df_clie04_["CALL_DATREF_MON"]).cumsum()
perc_sla_cum       = (slas_fechados_cum / total_de_calls_cum) * 100
perc_sla_cum_df    = pd.DataFrame(perc_sla_cum.values, columns = ["PERC_SLAS_CUM"], index = perc_sla_cum.index)

df_clie04_ = pd.merge(df_clie04_, perc_sla_cum_df, left_index = True, right_index = True)

# Criação da ABT de modelagem
ABT_clie04 = df_clie04_.drop(["customerCode", "CALL_DATREF_MON", "NUM_CLOSED_SLAS", "TOTAL_CALLS"], axis = 1)
ABT_clie04.head()

,PERC_SLAS_CUM
CALL_DATREF_INFO,
20190102,37.837838
20190103,44.382022
20190104,43.462898
20190105,45.205479
20190107,45.910290


### Cliente 5 - ID: 39159

In [19]:
# Cálculo da quantidade de chamados fechados dentro do SLA e o total de chamados por dia
slas_fechados     = df_clie05["FLAG_SLA"].groupby(df_clie05["CALL_DATREF_INFO"]).sum()
total_de_calls    = df_clie05["FLAG_SLA"].groupby(df_clie05["CALL_DATREF_INFO"]).count()
slas_fechados_df  = pd.DataFrame(slas_fechados.values, columns = ["NUM_CLOSED_SLAS"], index = slas_fechados.index)
total_de_calls_df = pd.DataFrame(total_de_calls.values, columns = ["TOTAL_CALLS"], index = total_de_calls.index)

df_aux     = pd.merge(slas_fechados_df, total_de_calls_df, left_index = True, right_index = True)

df_clie05.set_index("CALL_DATREF_INFO", drop = True, inplace = True)
df_clie05.drop("FLAG_SLA", axis = 1, inplace = True)

df_clie05_ = pd.merge(df_clie05, df_aux, left_on = "CALL_DATREF_INFO", right_on = "CALL_DATREF_INFO")
df_clie05_.drop_duplicates(inplace = True)

# Criação do percentual cumulativo de SLAS até o instante t
slas_fechados_cum  = df_clie05_["NUM_CLOSED_SLAS"].groupby(df_clie05_["CALL_DATREF_MON"]).cumsum()
total_de_calls_cum = df_clie05_["TOTAL_CALLS"].groupby(df_clie05_["CALL_DATREF_MON"]).cumsum()
perc_sla_cum       = (slas_fechados_cum / total_de_calls_cum) * 100
perc_sla_cum_df    = pd.DataFrame(perc_sla_cum.values, columns = ["PERC_SLAS_CUM"], index = perc_sla_cum.index)

df_clie05_ = pd.merge(df_clie05_, perc_sla_cum_df, left_index = True, right_index = True)

# Criação da ABT de modelagem
ABT_clie05 = df_clie05_.drop(["customerCode", "CALL_DATREF_MON", "NUM_CLOSED_SLAS", "TOTAL_CALLS"], axis = 1)
ABT_clie05.head()

,PERC_SLAS_CUM
CALL_DATREF_INFO,
20190103,100.000000
20190105,100.000000
20190107,90.909091
20190108,90.909091
20190109,94.594595


### Cliente 6 - ID: 87520

In [20]:
# Cálculo da quantidade de chamados fechados dentro do SLA e o total de chamados por dia
slas_fechados     = df_clie06["FLAG_SLA"].groupby(df_clie06["CALL_DATREF_INFO"]).sum()
total_de_calls    = df_clie06["FLAG_SLA"].groupby(df_clie06["CALL_DATREF_INFO"]).count()
slas_fechados_df  = pd.DataFrame(slas_fechados.values, columns = ["NUM_CLOSED_SLAS"], index = slas_fechados.index)
total_de_calls_df = pd.DataFrame(total_de_calls.values, columns = ["TOTAL_CALLS"], index = total_de_calls.index)

df_aux     = pd.merge(slas_fechados_df, total_de_calls_df, left_index = True, right_index = True)

df_clie06.set_index("CALL_DATREF_INFO", drop = True, inplace = True)
df_clie06.drop("FLAG_SLA", axis = 1, inplace = True)

df_clie06_ = pd.merge(df_clie06, df_aux, left_on = "CALL_DATREF_INFO", right_on = "CALL_DATREF_INFO")
df_clie06_.drop_duplicates(inplace = True)

# Criação do percentual cumulativo de SLAS até o instante t
slas_fechados_cum  = df_clie06_["NUM_CLOSED_SLAS"].groupby(df_clie06_["CALL_DATREF_MON"]).cumsum()
total_de_calls_cum = df_clie06_["TOTAL_CALLS"].groupby(df_clie06_["CALL_DATREF_MON"]).cumsum()
perc_sla_cum       = (slas_fechados_cum / total_de_calls_cum) * 100
perc_sla_cum_df    = pd.DataFrame(perc_sla_cum.values, columns = ["PERC_SLAS_CUM"], index = perc_sla_cum.index)

df_clie06_ = pd.merge(df_clie06_, perc_sla_cum_df, left_index = True, right_index = True)

# Criação da ABT de modelagem
ABT_clie06 = df_clie06_.drop(["customerCode", "CALL_DATREF_MON", "NUM_CLOSED_SLAS", "TOTAL_CALLS"], axis = 1)
ABT_clie06.head()

,PERC_SLAS_CUM
CALL_DATREF_INFO,
20190102,71.264368
20190103,72.222222
20190104,71.345029
20190105,72.960373
20190106,74.946004


### Cliente 7 - ID: 372307

In [21]:
# Cálculo da quantidade de chamados fechados dentro do SLA e o total de chamados por dia
slas_fechados     = df_clie07["FLAG_SLA"].groupby(df_clie07["CALL_DATREF_INFO"]).sum()
total_de_calls    = df_clie07["FLAG_SLA"].groupby(df_clie07["CALL_DATREF_INFO"]).count()
slas_fechados_df  = pd.DataFrame(slas_fechados.values, columns = ["NUM_CLOSED_SLAS"], index = slas_fechados.index)
total_de_calls_df = pd.DataFrame(total_de_calls.values, columns = ["TOTAL_CALLS"], index = total_de_calls.index)

df_aux     = pd.merge(slas_fechados_df, total_de_calls_df, left_index = True, right_index = True)

df_clie07.set_index("CALL_DATREF_INFO", drop = True, inplace = True)
df_clie07.drop("FLAG_SLA", axis = 1, inplace = True)

df_clie07_ = pd.merge(df_clie07, df_aux, left_on = "CALL_DATREF_INFO", right_on = "CALL_DATREF_INFO")
df_clie07_.drop_duplicates(inplace = True)

# Criação do percentual cumulativo de SLAS até o instante t
slas_fechados_cum  = df_clie07_["NUM_CLOSED_SLAS"].groupby(df_clie07_["CALL_DATREF_MON"]).cumsum()
total_de_calls_cum = df_clie07_["TOTAL_CALLS"].groupby(df_clie07_["CALL_DATREF_MON"]).cumsum()
perc_sla_cum       = (slas_fechados_cum / total_de_calls_cum) * 100
perc_sla_cum_df    = pd.DataFrame(perc_sla_cum.values, columns = ["PERC_SLAS_CUM"], index = perc_sla_cum.index)

df_clie07_ = pd.merge(df_clie07_, perc_sla_cum_df, left_index = True, right_index = True)

# Criação da ABT de modelagem
ABT_clie07 = df_clie07_.drop(["customerCode", "CALL_DATREF_MON", "NUM_CLOSED_SLAS", "TOTAL_CALLS"], axis = 1)
ABT_clie07.head()

,PERC_SLAS_CUM
CALL_DATREF_INFO,
20190101,83.333333
20190102,59.160305
20190103,62.407407
20190104,62.874251
20190105,62.985685


### Cliente 8 - ID: 797391

In [22]:
# Cálculo da quantidade de chamados fechados dentro do SLA e o total de chamados por dia
slas_fechados     = df_clie08["FLAG_SLA"].groupby(df_clie08["CALL_DATREF_INFO"]).sum()
total_de_calls    = df_clie08["FLAG_SLA"].groupby(df_clie08["CALL_DATREF_INFO"]).count()
slas_fechados_df  = pd.DataFrame(slas_fechados.values, columns = ["NUM_CLOSED_SLAS"], index = slas_fechados.index)
total_de_calls_df = pd.DataFrame(total_de_calls.values, columns = ["TOTAL_CALLS"], index = total_de_calls.index)

df_aux     = pd.merge(slas_fechados_df, total_de_calls_df, left_index = True, right_index = True)

df_clie08.set_index("CALL_DATREF_INFO", drop = True, inplace = True)
df_clie08.drop("FLAG_SLA", axis = 1, inplace = True)

df_clie08_ = pd.merge(df_clie08, df_aux, left_on = "CALL_DATREF_INFO", right_on = "CALL_DATREF_INFO")
df_clie08_.drop_duplicates(inplace = True)

# Criação do percentual cumulativo de SLAS até o instante t
slas_fechados_cum  = df_clie08_["NUM_CLOSED_SLAS"].groupby(df_clie08_["CALL_DATREF_MON"]).cumsum()
total_de_calls_cum = df_clie08_["TOTAL_CALLS"].groupby(df_clie08_["CALL_DATREF_MON"]).cumsum()
perc_sla_cum       = (slas_fechados_cum / total_de_calls_cum) * 100
perc_sla_cum_df    = pd.DataFrame(perc_sla_cum.values, columns = ["PERC_SLAS_CUM"], index = perc_sla_cum.index)

df_clie08_ = pd.merge(df_clie08_, perc_sla_cum_df, left_index = True, right_index = True)

# Criação da ABT de modelagem
ABT_clie08 = df_clie08_.drop(["customerCode", "CALL_DATREF_MON", "NUM_CLOSED_SLAS", "TOTAL_CALLS"], axis = 1)
ABT_clie08.head()

,PERC_SLAS_CUM
CALL_DATREF_INFO,
20190111,100.000000
20190114,100.000000
20190115,100.000000
20190123,85.714286
20190201,100.000000


### Cliente 9 - ID: 900102

* Obs: Cliente não possui dados para a referência a ser desenvolvida o modelo

### Cliente 10 - ID: 900995

In [23]:
# Cálculo da quantidade de chamados fechados dentro do SLA e o total de chamados por dia
slas_fechados     = df_clie10["FLAG_SLA"].groupby(df_clie10["CALL_DATREF_INFO"]).sum()
total_de_calls    = df_clie10["FLAG_SLA"].groupby(df_clie10["CALL_DATREF_INFO"]).count()
slas_fechados_df  = pd.DataFrame(slas_fechados.values, columns = ["NUM_CLOSED_SLAS"], index = slas_fechados.index)
total_de_calls_df = pd.DataFrame(total_de_calls.values, columns = ["TOTAL_CALLS"], index = total_de_calls.index)

df_aux     = pd.merge(slas_fechados_df, total_de_calls_df, left_index = True, right_index = True)

df_clie10.set_index("CALL_DATREF_INFO", drop = True, inplace = True)
df_clie10.drop("FLAG_SLA", axis = 1, inplace = True)

df_clie10_ = pd.merge(df_clie10, df_aux, left_on = "CALL_DATREF_INFO", right_on = "CALL_DATREF_INFO")
df_clie10_.drop_duplicates(inplace = True)

# Criação do percentual cumulativo de SLAS até o instante t
slas_fechados_cum  = df_clie10_["NUM_CLOSED_SLAS"].groupby(df_clie10_["CALL_DATREF_MON"]).cumsum()
total_de_calls_cum = df_clie10_["TOTAL_CALLS"].groupby(df_clie10_["CALL_DATREF_MON"]).cumsum()
perc_sla_cum       = (slas_fechados_cum / total_de_calls_cum) * 100
perc_sla_cum_df    = pd.DataFrame(perc_sla_cum.values, columns = ["PERC_SLAS_CUM"], index = perc_sla_cum.index)

df_clie10_ = pd.merge(df_clie10_, perc_sla_cum_df, left_index = True, right_index = True)

# Criação da ABT de modelagem
ABT_clie10 = df_clie10_.drop(["customerCode", "CALL_DATREF_MON", "NUM_CLOSED_SLAS", "TOTAL_CALLS"], axis = 1)
ABT_clie10.head()

,PERC_SLAS_CUM
CALL_DATREF_INFO,
20190102,100.000000
20190103,75.000000
20190104,80.000000
20190107,88.888889
20190108,90.000000


## Exportação das tabelas de modelagem com alterações na chave para date

In [24]:
# Ajuste na chave para deixar em formato date AAAA-MM-DD
ABT_clie01.reset_index(drop = False, inplace = True)
ABT_clie01["DT_DATREF_INFO"] = [date(x//10**4, (x%10**4)//100, x%10**2) for x in ABT_clie01["CALL_DATREF_INFO"]]
ABT_clie01.drop("CALL_DATREF_INFO", axis = 1, inplace = True)
ABT_clie01.set_index("DT_DATREF_INFO", drop = True, inplace = True)

ABT_clie02.reset_index(drop = False, inplace = True)
ABT_clie02["DT_DATREF_INFO"] = [date(x//10**4, (x%10**4)//100, x%10**2) for x in ABT_clie02["CALL_DATREF_INFO"]]
ABT_clie02.drop("CALL_DATREF_INFO", axis = 1, inplace = True)
ABT_clie02.set_index("DT_DATREF_INFO", drop = True, inplace = True)

ABT_clie03.reset_index(drop = False, inplace = True)
ABT_clie03["DT_DATREF_INFO"] = [date(x//10**4, (x%10**4)//100, x%10**2) for x in ABT_clie03["CALL_DATREF_INFO"]]
ABT_clie03.drop("CALL_DATREF_INFO", axis = 1, inplace = True)
ABT_clie03.set_index("DT_DATREF_INFO", drop = True, inplace = True)

ABT_clie04.reset_index(drop = False, inplace = True)
ABT_clie04["DT_DATREF_INFO"] = [date(x//10**4, (x%10**4)//100, x%10**2) for x in ABT_clie04["CALL_DATREF_INFO"]]
ABT_clie04.drop("CALL_DATREF_INFO", axis = 1, inplace = True)
ABT_clie04.set_index("DT_DATREF_INFO", drop = True, inplace = True)

ABT_clie05.reset_index(drop = False, inplace = True)
ABT_clie05["DT_DATREF_INFO"] = [date(x//10**4, (x%10**4)//100, x%10**2) for x in ABT_clie05["CALL_DATREF_INFO"]]
ABT_clie05.drop("CALL_DATREF_INFO", axis = 1, inplace = True)
ABT_clie05.set_index("DT_DATREF_INFO", drop = True, inplace = True)

ABT_clie06.reset_index(drop = False, inplace = True)
ABT_clie06["DT_DATREF_INFO"] = [date(x//10**4, (x%10**4)//100, x%10**2) for x in ABT_clie06["CALL_DATREF_INFO"]]
ABT_clie06.drop("CALL_DATREF_INFO", axis = 1, inplace = True)
ABT_clie06.set_index("DT_DATREF_INFO", drop = True, inplace = True)

ABT_clie07.reset_index(drop = False, inplace = True)
ABT_clie07["DT_DATREF_INFO"] = [date(x//10**4, (x%10**4)//100, x%10**2) for x in ABT_clie07["CALL_DATREF_INFO"]]
ABT_clie07.drop("CALL_DATREF_INFO", axis = 1, inplace = True)
ABT_clie07.set_index("DT_DATREF_INFO", drop = True, inplace = True)

ABT_clie08.reset_index(drop = False, inplace = True)
ABT_clie08["DT_DATREF_INFO"] = [date(x//10**4, (x%10**4)//100, x%10**2) for x in ABT_clie08["CALL_DATREF_INFO"]]
ABT_clie08.drop("CALL_DATREF_INFO", axis = 1, inplace = True)
ABT_clie08.set_index("DT_DATREF_INFO", drop = True, inplace = True)

ABT_clie10.reset_index(drop = False, inplace = True)
ABT_clie10["DT_DATREF_INFO"] = [date(x//10**4, (x%10**4)//100, x%10**2) for x in ABT_clie10["CALL_DATREF_INFO"]]
ABT_clie10.drop("CALL_DATREF_INFO", axis = 1, inplace = True)
ABT_clie10.set_index("DT_DATREF_INFO", drop = True, inplace = True)

In [25]:
# Exportação dos dados
ABT_clie01.to_csv("sla_abt_clie01.csv")
ABT_clie02.to_csv("sla_abt_clie02.csv")
ABT_clie03.to_csv("sla_abt_clie03.csv")
ABT_clie04.to_csv("sla_abt_clie04.csv")
ABT_clie05.to_csv("sla_abt_clie05.csv")
ABT_clie06.to_csv("sla_abt_clie06.csv")
ABT_clie07.to_csv("sla_abt_clie07.csv")
ABT_clie08.to_csv("sla_abt_clie08.csv")
ABT_clie10.to_csv("sla_abt_clie10.csv")